# Implementation of Logistic Regression on AMP-PD 

#### Authors: Maria Castanos and William Koehler

## Introduction
The objective of this analysis is to train a regularized logistic regression on the AMP-PD Dataset. However, to achieve computational efficiency, feature selecion is performed to reduce the dataset's dimensions, while maintaining the  necessary information to achieve the highest prediciton power. 

## Problem Description
This analysis uses statistical tools to use gene variants to classify patients with Parkinson's Disease.

### Dataset
The dataset has a total of 2790 patients, out of which 1711 are diagnosed with Parkinson's. There is a total of 1,981,373 gene variants per patient. 

Gene variants can have the value of 0, 1, or 2, which mean the patient is homozygous for first allele, homozygous for second allele, and heterozygous, respectively.

### Feature Selection
[Various feature selection methods](https://scikit-learn.org/stable/modules/feature_selection.html) are performed to score the realtionships between the predictors and the dependent variable. Dimensionality reduction is then performed by picking the $k$ predictors with the highest $k$ scores. Specifically, relationships are scored using the methods described below.

#### Tree Based Selection 
This method computes impurity-based feature importances to drop irrelevant features. 

#### Univariate Feature Selection
This method chooses the best features based on univariate statistical tests that score the degree of dependency between the predictors and the dependent variable. The tests used where:
 - Chi-squared
 - ANOVA F-value
 - Mutual Information Score

## Training and Tuning 
For each feature selection method, the training was done on the top $100$, $1000$, and $10000$ scored features.

### Regularized Logistic Regression 
A regularized multinomial logistic regression is trained to predict two classes (Control, Case, Other). Elastic Net penalization was used in order to find a compromise between ridge and lasso penalizations. 

The optimization problem is:
$$\max_{\beta_{0k}, \beta_{k}} \left\{ \sum_{i = 1}^{N} \log Pr(g_i|x_i) - \lambda \sum_{k = 1}^K\sum_{j = 1}^p \big(\alpha |\beta_{kj}| + (1 - \alpha)\beta_{kj}^2\big) \right\}$$ 

#### Hyperparameter Optimization
Hyperparameters $\lambda$ and $\alpha$ in the problem above, are optimized by implementing a Stratified ShuffleSplit cross-validator. The optimal hyperparametrs are those that maximize the balanced accuracy:
$$\frac{1}{2}\left( \frac{TP}{TP + FN} + \frac{TN}{FP + TN} \right)$$

## Experiments
The experiments described above were performed as described bellow.
### Save and Load Data
In order to construct the $X$ matrix containing the gene variants the [.pgen](https://s3.console.aws.amazon.com/s3/object/amp-pd-data?region=us-west-2&prefix=genomic-data/reduced/ldpruned_data.pgen) ( PLINK 2's preferred way to represent genotype calls) and [.psam](https://s3.console.aws.amazon.com/s3/object/amp-pd-data?region=us-west-2&prefix=genomic-data/reduced/ldpruned_data.psam) (sample information). The $y$ vector was constructed using the [labels file](https://s3.console.aws.amazon.com/s3/object/amp-pd-data?region=us-west-2&prefix=genomic-data/reduced/latest_labels.tsv). 

The [train_test_split.py](https://github.com/occamzrazor/genoml2/blob/feature/train_models/genoml/razor_training/train_test_split.py) script was ran to generate and save to disk the training and test splits for both the predictive variables ($X$ matrix) and the dependent variable ($y$ vector).

### Select Features

#### Tree Based Selection
First we do feature selection with a tree based method to select the most important k features, according to the  impurity-based feature importances: 

#### Univariate Chi-Squared
Similarly to the tree based method, we do feature selection with Chi-squared univariate tests to select the most important k features, according to these test scores: 

#### Univariate ANOVA F-value

### Train Logistic Regression 
For each method, we train the logistic regression on the selected features and save the resulting models to disk by running:

## Results
Various performance metrics for classification tasks are used to evaluate the results of the experiments described above. In particular:

- *Accuracy:* Ratio between the number of correctly classified patients to the total number of patients.
- *Precision:* Ratio between the correctly classified Parkinson's patients to the total classified Parkinson's patients.
- *Recall:* Ratio between the correctly classified Parkinson's patients to the total patients with Parkinson's disease on the dataset.
- *F1-Score:* Harmonic mean of precision and recall. It balances both the concerns of precision and recall in one number.
- *Balanced Accuracy:* It's the average of the within class accuracy.

In [1]:
from results import results

In [3]:
%%capture
performance_table = results.performance_table()

In [4]:
performance_table.index = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Balanced Accuracy'] 
performance_table.round(3)

,Tree 100,Tree 1000,Tree 10000,Chi2 100,Chi2 1000,Chi2 10000,Ftest 100,Ftest 1000,Ftest 10000,Random,Majority Class
Accuracy,0.750,0.726,0.688,0.745,0.740,0.695,0.749,0.728,0.694,0.576,0.481
Precision,0.791,0.778,0.729,0.785,0.789,0.725,0.787,0.779,0.716,0.576,0.556
Recall,0.768,0.733,0.729,0.768,0.749,0.756,0.772,0.737,0.776,1.000,0.497
F1-Score,0.779,0.755,0.729,0.776,0.768,0.740,0.780,0.758,0.745,0.731,0.525
Balanced Accuracy,0.746,0.725,0.681,0.741,0.738,0.684,0.744,0.727,0.679,0.500,0.479


Since our dataset is somewhat balanced, accuracy is a reasonable metric to evaluate. However, since the number positives and the number of negatives is still different, Balanced Accuracy should be observed more in detail.
We observe that as more predictors are added into the models, both the *Accuracy* and the *Balanced Accuracy* metrics deteriorate.

For $k = 100$, the tree-based method yields the highest *Accuracy* and *Balanced Accuracy* the other methods on every metric; however, results are don't differ much. The tree-based method (with $k = 100$), also outpreforms other methods on precision, recall, and f1-score. 

### Most Important Features
We are also interested in outputting the coeffitients of the selected features in order to see the impact on the log odds ratio. In particular, we show the column numbers ordered from highest to lower coefficient. 
#### Tree-Based Method (k = 100)

In [8]:
import numpy as np

In [15]:
coeff = results.get_coefficients('tree', 100)
np.argsort(-np.abs(coeff))

array([[17, 79, 83, 35,  1, 10, 28, 41, 11,  3,  0, 98,  2, 92, 76,  6,
        70, 93, 64, 31, 61, 12,  5, 40, 36, 95, 39, 63, 94, 19, 23,  8,
        42, 71, 24, 86, 13, 67, 59,  7, 29, 38,  9, 34, 49, 25, 47, 32,
        51, 20, 30, 62, 43, 27, 14, 26, 90, 85, 15, 21, 45, 22, 69, 81,
        82, 46, 72,  4, 56, 65, 44, 68, 48, 74, 33, 99, 57, 54, 50, 84,
        88, 75, 16, 66, 91, 60, 58, 97, 87, 78, 37, 89, 18, 96, 52, 77,
        80, 53, 55, 73]])

#### Univariate Chi-Squared (k = 100)

In [14]:
coeff = results.get_coefficients('univariate_chi2', 100)
np.argsort(-np.abs(coeff))

array([[55, 89, 77,  0, 32, 65, 97, 34, 78, 69, 44,  4, 58,  6, 37, 73,
        80, 57, 49, 94, 62, 28,  2, 66, 51, 23, 26,  8, 88, 60, 90,  1,
        15, 48, 13, 46, 82, 29, 85, 50, 81, 59, 11, 25, 40, 45, 35, 27,
        96, 24, 18,  5, 20, 14, 54, 91, 31, 71, 67, 10, 86, 76, 30, 68,
        36, 52, 38, 16, 99, 56, 93, 70, 43, 12, 41, 61,  9, 64, 74,  3,
        19, 17, 42, 47, 72, 39,  7, 92, 98, 22, 63, 33, 53, 95, 83, 84,
        87, 21, 79, 75]])

#### Univariate ANOVA F-Value (k = 100)

In [13]:
coeff = results.get_coefficients('univariate_fclassif', 100)
np.argsort(-np.abs(coeff))

array([[ 2,  0,  4,  5,  3,  1, 18, 19,  6, 11, 10, 26,  9,  8, 13, 45,
        32, 20, 22, 23, 48, 24, 41,  7, 38, 62, 16, 21, 37, 49, 27, 28,
        40, 70, 17, 15, 36, 66, 98, 47, 99, 43, 30, 14, 39, 67, 71, 68,
        58, 12, 25, 53, 78, 75, 79, 81, 73, 94, 44, 91, 29, 77, 93, 60,
        33, 31, 55, 46, 92, 63, 83, 84, 51, 89, 50, 74, 85, 72, 34, 35,
        56, 80, 95, 65, 88, 57, 52, 86, 76, 87, 54, 59, 82, 96, 90, 61,
        69, 42, 64, 97]])

### PCA
Since our goal is to achieve the most compact and poweful combination of features, we explored the implementation of PCA on the reduced datasets and then perform logistic regression on the resulting datasets. 

The results of such experiments aree displayed below:

In [5]:
performance_table = results.performance_table(do_pca = 'do_pca')
performance_table.index = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Balanced Accuracy'] 
performance_table.round(3)

,Tree 100,Tree 1000,Tree 10000,Chi2 100,Chi2 1000,Chi2 10000,Ftest 100,Ftest 1000,Ftest 10000,Random,Majority Class
Accuracy,0.727,0.722,0.687,0.740,0.740,0.654,0.749,0.726,0.687,0.576,0.513
Precision,0.777,0.774,0.731,0.780,0.787,0.705,0.788,0.782,0.716,0.576,0.589
Recall,0.739,0.729,0.721,0.762,0.750,0.686,0.770,0.727,0.756,1.000,0.509
F1-Score,0.757,0.751,0.726,0.771,0.768,0.696,0.779,0.754,0.736,0.731,0.546
Balanced Accuracy,0.725,0.720,0.681,0.736,0.738,0.649,0.745,0.726,0.674,0.500,0.514


When projecting the data onto a lower space, results change very little. This can mean that $95\%$ of the variance contains most of the power to predict $y$. For each feature selection method and each k, we explain how PCA reduces dimensions below.

For $k = 100$, the Univariate method with ANOVA F-values outperforms the others on every metric.

#### Tree
When performing PCA on the top k features,  $k = 100$, picked by the tree-based method, the dataset is reduced to $80$ features by PCA.

For $k = 1000$, the dataset is reduced to $674$ features by PCA.

For $k = 10000$, the dataset is reduced to $2369$ features by PCA.

#### Univariate Chi-Squared
When performing PCA on the top k features,  $k = 100$, picked by the chi-squared tests, the dataset is reduced to $83$ features by PCA.

For $k = 1000$, the dataset is reduced to $687$ features by PCA.

For $k = 10000$, the dataset is reduced to $2302$ features by PCA.

#### Univariate ANOVA F-Value
When performing PCA on the top k features,  $k = 100$, picked by the F-tests, the dataset is reduced to $82$ features by PCA.

For $k = 1000$, the dataset is reduced to $716$ features by PCA.

For $k = 10000$, the dataset is reduced to $2370$ features by PCA.

This can be verified with the [files generated](https://s3.console.aws.amazon.com/s3/buckets/amp-pd-data?region=us-west-2&prefix=genomic-data/reduced/results/PCA/&showversions=false) from running the [experiments_pca.py](https://github.com/occamzrazor/genoml2/blob/feature/train_models/genoml/razor_training/experiments_pca.py) script.

## Notes
The results presented above can also be replicated by running, the [experiments.py](https://github.com/occamzrazor/genoml2/blob/feature/train_models/genoml/razor_training/experiments.py) and [results.py](https://github.com/occamzrazor/genoml2/blob/feature/train_models/genoml/razor_training/results.py) scripts. To generate the table with the PCA results, run [experiments_pca.py](https://github.com/occamzrazor/genoml2/blob/feature/train_models/genoml/razor_training/experiments_pca.py) (after running [experiments.py](https://github.com/occamzrazor/genoml2/blob/feature/train_models/genoml/razor_training/experiments.py)) and then run [results.py](https://github.com/occamzrazor/genoml2/blob/feature/train_models/genoml/razor_training/results.py) script with do_pca='do_pca'.

However, these scripts were refactored into the [logistic_regression_eexperiments.py](https://github.com/occamzrazor/genoml2/blob/feature/logreg_experiments/genoml/razor_training/logistic_regression_experiments.py) script, and its usage is described in the Experiments Section of this Notebook.

Mutual Information Score was not implementeed due to a lack of computational power. In the future, if we want to add this analysis, a bigger machine should be used to run.